# ROI detection with Deep Learning
Notebook used to test ROI detection using PyTorch

In [1]:
%matplotlib inline

import os, sys, time, shutil
import random
import ipywidgets as widgets
from ipywidgets import interact

import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torchvision
from torch.utils import data

from utils_common.image import imread_to_float, to_npint
from utils_data import normalize_range, get_all_dataloaders, get_filenames
from utils_loss import get_crop_loss, get_dice_metric, get_crop_dice_metric
from utils_model import weights_initialization, CustomUNet
from utils_train import train
from utils_test import predict, predict_stack, evaluate, show_sample

seed = 1
random.seed(seed)
np.random.seed(seed*10 + 1234)
torch.manual_seed(seed*100 + 4321)

# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)

%load_ext autoreload
%autoreload 2

Device: cuda:0


## Parameters
Hyperparameters, folder names, etc.

In [2]:
n_epochs = 2
batch_size = 32
learning_rate = 0.001
diceC_scale = 4.0 # scale of the cropping (w.r.t. ROI's bounding box) for the dice coef.
synth_data = True
synth_ratio = 0.5 # ratio of synthetic data vs. real data
only_synth = True # If True, will use only the synthetic data (and all of it, at the opposite of ratio=1)

input_channels = "R" # Channel to use as input
u_depth = 4
out1_channels = 16

model_name = "test"
model_dir = "models/"
data_dir = "../dataset/"

## Data loading
Create dataloaders, and build data-related constants.

In [3]:
# Create transforms that pad images to multiple of 2 ** u_depth if needed
def pad_transform(image):
    factor = 2 ** u_depth
    if image.ndim == 3:
        height, width = image.shape[1:]
    elif image.ndim == 2:
        height, width = image.shape
        
    # Do nothing if image has correct shape
    if height % factor == 0 and width % factor == 0:
        return image
    
    height_pad = (factor - height % factor) * bool(height % factor)
    width_pad = (factor - width % factor) * bool(width % factor)
    padding = [(int(np.floor(height_pad/2)), int(np.ceil(height_pad/2))), 
               (int(np.floor(width_pad/2)), int(np.ceil(width_pad/2)))]
    if image.ndim == 3:
        return np.pad(image, [(0,0)] + padding, 'constant')
    elif image.ndim == 2:
        return np.pad(image, padding, 'constant')

# Create dataloaders
dataloaders = get_all_dataloaders(
    data_dir,
    batch_size, 
    input_channels = input_channels,
    test_dataloader = True,
    synthetic_data = synth_data, synthetic_ratio = synth_ratio,
    synthetic_only = only_synth,
    train_transform = pad_transform, train_target_transform = pad_transform,
    eval_transform = pad_transform, eval_target_transform = pad_transform
)

N_TRAIN = len(dataloaders["train"].dataset)
N_VALID = len(dataloaders["valid"].dataset)
N_TEST = len(dataloaders["test"].dataset)
HEIGHT, WIDTH = io.imread(get_filenames(os.path.join(data_dir, "train/"))[0][0]).shape[:2]
print("%d images of size %dx%d." % (N_TRAIN + N_VALID + N_TEST, HEIGHT, WIDTH))
print("%d to train" % N_TRAIN, end="")
if only_synth:
    print(" (of synthetic data only).")
elif synth_data:
    if synth_ratio is None:
        print(" (with synthetic data)")
    else:
        print(" (with %d%% of synthetic data)" % (synth_ratio * 100))
else:
    print("")
print("%d to validation" % N_VALID)
print("%d to test" % N_TEST)

# Compute class weights (as pixel imbalance) takes ~90s for 78 stacks
pos_count = 0
neg_count = 0
# for _, batch_y in dataloaders["train"]:
#     pos_count += (batch_y == 1).sum().item()
#     neg_count += (batch_y == 0).sum().item()
# pos_weight = torch.tensor(neg_count / pos_count).to(device)
# Following has been pre-computed
pos_weight = torch.tensor(120.0).to(device)
if pos_count != 0 and neg_count != 0:
    print("{} ROI pixels, and {} background --> {:f} positive weighting.".format(
        pos_count, neg_count, pos_weight.item()))
else:
    print("{:.3f} positive weighting.".format(pos_weight.item()))

71000 images of size 192x256.
40000 to train (of synthetic data only).
14100 to validation
16900 to test
120.000 positive weighting.


## Model definition
Define model, loss, metrics, and optimizer.

In [4]:
model = CustomUNet(len(input_channels), u_depth=u_depth, 
                   out1_channels=out1_channels, batchnorm=True, device=device)
print(model)

# Save the "architecture" of the model by copy/pasting the class definition file
os.makedirs(os.path.join(model_dir, model_name), exist_ok=True)
shutil.copy("utils_model.py", os.path.join(model_dir, model_name, "utils_model_save.py"))

loss_fn = torch.nn.BCEWithLogitsLoss(reduction='elementwise_mean', pos_weight=pos_weight)
crop_loss = get_crop_loss(loss_fn, scale=diceC_scale, device=device)

dice_metric = get_dice_metric()
diceC_metric = get_crop_dice_metric(scale=diceC_scale)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

CustomUNet(
  (activation): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (convs): ModuleList(
    (0): UNetConv(
      (activation): ReLU()
      (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): UNetConv(
      (activation): ReLU()
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): UNetConv(
      (activation): ReLU()
      (co

## Training

In [5]:
best_model, history = train(model,
                            dataloaders,
                            loss_fn,
                            optimizer,
                            n_epochs,
                            metrics = {"lossC%.1f"%diceC_scale: crop_loss, 
                                       "dice": dice_metric, "diC%.1f"%diceC_scale: diceC_metric},
                            criterion_metric = "dice",
                            model_dir = os.path.join(model_dir, model_name),
                            replace_dir = True,
                            
                            verbose = 1)

Epoch 1/2  (Elapsed time: 0h 00min 00s)
---------------------------------------
Batch (over 1250): 1...125...250...375...

Process Process-1:
Traceback (most recent call last):
  File "/home/user/anaconda3/envs/talabot/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/user/anaconda3/envs/talabot/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/user/anaconda3/envs/talabot/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/user/anaconda3/envs/talabot/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/user/talabot/ROI_detection/deep_learning/utils_data.py", line 58, in __getitem__
    image = imread_to_float(self.x_filenames[idx], scaling=255)
  File "/home/user/talabot/ROI_detection/deep_learning/utils_common/image.py", line 16, in imread_to_float
    stack = io.imread(filenam

KeyboardInterrupt: 

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.subplot(121)
plt.title("Loss\n(crop scale = %.1f)" % diceC_scale)
plt.plot(history["epoch"], history["loss"], color="C0")
plt.plot(history["epoch"], history["lossC%.1f" % diceC_scale], "--", color="C0")
plt.plot(history["epoch"], history["val_loss"], color="C1")
plt.plot(history["epoch"], history["val_lossC%.1f" % diceC_scale], "--", color="C1")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(["train loss", "train cropped loss", "valid loss", "valid cropped loss"])
plt.subplot(122)
plt.title("Dice coefficients\n(crop scale = %.1f)" % diceC_scale)
plt.plot(history["epoch"], history["dice"], color="C0")
plt.plot(history["epoch"], history["diC%.1f" % diceC_scale], "--", color="C0")
plt.plot(history["epoch"], history["val_dice"], color="C1")
plt.plot(history["epoch"], history["val_diC%.1f" % diceC_scale], "--", color="C1")
plt.xlabel("Epoch")
plt.ylabel("Dice coef.")
plt.ylim(0,1)
plt.legend(["train dice", "train cropped dice", "valid dice", "valid cropped dice"])
# plt.subplot(133)
# plt.title("Cropped Dice coefficient (scale = %.1f)" % diceC_scale)
# plt.plot(history["epoch"], history["diC%.1f"%diceC_scale])
# plt.plot(history["epoch"], history["val_diC%.1f"%diceC_scale])
# plt.xlabel("Epoch")
# plt.ylabel("Cropped Dice coef.")
# plt.ylim(0,1)
# plt.legend(["train diC%.1f"%diceC_scale, "valid diC%.1f"%diceC_scale])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## Testing
Test and visualize best model on valid/test data. Note that this is for debugging purposes: you should use the test data only at the very end! Otherwise, just use validation data.

In [ ]:
show_sample(best_model, dataloaders["valid"], n_samples=4, 
            post_processing = lambda preds: torch.tensor(torch.sigmoid(preds) > 0.5, dtype=torch.float32), 
            metrics = ({"lossC%.1f"%diceC_scale: crop_loss, 
                        "dice": dice_metric, "diC%.1f"%diceC_scale: diceC_metric}))

In [ ]:
show_sample(best_model, dataloaders["valid"], n_samples=4, 
            post_processing = None, 
            metrics = ({"lossC%.1f"%diceC_scale: crop_loss, 
                        "dice": dice_metric, "diC%.1f"%diceC_scale: diceC_metric}))

### Test on a full stack with no ground truth.

In [ ]:
rgb_stack = imread_to_float(
    "/home/user/talabot/workdir/20180716_R85A02-tdTomGC6fopt-fly4/R85A02-tdTomGC6fopt-fly4-001/RGB.tif")
channels = {"R": rgb_stack[...,0], "G": rgb_stack[...,1], "B": rgb_stack[...,2]}
stack = np.stack([channels[channel] for channel in input_channels], axis=1)

start_predict = time.time()
predictions = predict_stack(best_model, stack, batch_size, input_channels=input_channels)
print("Prediction took %.1fs." % (time.time() - start_predict))

@interact(num = widgets.BoundedIntText(min=0, max=len(rgb_stack) - 1, description='image:'))
def plot_stack_img(num=0):
    plt.figure(figsize=(12,12))
    plt.subplot(221)
    plt.title("RGB data")
    plt.imshow(rgb_stack[num])
    plt.subplot(222)
    plt.title("Input")
    if input_channels == "R":
        plt.imshow(rgb_stack[num,...,0], vmin=0, vmax=1, cmap="gray")
    elif input_channels == "RG":
        plt.imshow(rgb_stack[num], vmin=0, vmax=1)
    plt.subplot(223)
    plt.title("Prediction")
    plt.imshow(predictions[num], vmin=0, vmax=1, cmap="gray")
    plt.subplot(224)
    plt.title("Binary reduction")
    plt.imshow(predictions[num] > 0.5, vmin=0, vmax=1, cmap="gray")
    plt.tight_layout()
    plt.show()